pca_fasttext_100features

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import warnings
warnings.filterwarnings("ignore") 

In [5]:
import os
import numpy as np 
import pandas as pd
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

In [6]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/PCA_Fasttext/data.csv"
df = pd.read_csv(path)
df.head() 

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,Level 5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,Level 5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",Level 5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [7]:
df.iloc[:,1].replace(['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5'],
                        [1, 2, 3, 4, 5], inplace=True)

In [8]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [9]:
#define bins
bins = [0,2,4,5]
#name of the group 
group_names = ['Low', 'Moderate', 'High']
df['Level '] = pd.cut(df["Level "],bins,labels = group_names)

In [10]:
df.head(5)

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,High,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,High,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,High,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",High,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Low,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [11]:
df['Level '] = df['Level '].map({'Low': 1, 'Moderate': 2, 'High': 3}).astype(int)

In [12]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,3,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,3,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,3,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",3,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,1,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [13]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/PCA_Fasttext/fasttext_withstopwordchangelevels_100_features.csv"
x = pd.read_csv(path)
x.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,0,-0.165540,-4.728341,-1.595213,2.680546,-5.212836,-2.252168,-2.662594,-0.349344,0.452097,...,-1.779400e-07,2.036389e-07,8.022884e-07,-2.538071e-07,1.863774e-06,4.875392e-07,3.212453e-07,6.347524e-07,6.815264e-07,7.782946e-07
1,1,10.305937,11.510157,-5.758058,-1.508590,-8.974911,-2.647319,-4.362800,4.050844,-0.830384,...,9.048996e-07,-5.735345e-07,-6.721415e-07,8.550529e-07,3.789117e-07,1.700441e-07,-3.308357e-07,3.262595e-07,8.682251e-07,1.890846e-06
2,2,-8.348694,-0.363074,0.076656,-1.005099,1.440403,-0.442519,-1.781688,-1.407007,-0.044553,...,4.977300e-07,8.914771e-07,-3.189962e-07,5.905976e-07,-7.565585e-07,-4.783064e-07,-1.011099e-06,1.576675e-06,-3.102626e-07,1.431196e-06
3,3,-1.383220,2.399253,-1.932302,3.060848,1.171404,-2.749633,0.390031,-4.638575,-0.636720,...,-2.019109e-07,7.114815e-07,3.220782e-07,4.626671e-08,3.813217e-08,1.569164e-06,-7.562271e-07,4.933081e-07,-3.309136e-07,1.466063e-07
4,4,-7.381315,1.570987,0.302512,-1.793322,-1.895764,1.705287,-2.147691,-0.547170,-0.804376,...,-8.186127e-07,9.035237e-08,5.769418e-07,-3.454535e-07,4.934978e-07,1.786420e-06,-5.246066e-07,2.030548e-07,-1.557495e-06,1.249375e-06


In [14]:
x.drop(x.columns[[0]], axis=1, inplace=True)
x.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.165540,-4.728341,-1.595213,2.680546,-5.212836,-2.252168,-2.662594,-0.349344,0.452097,-0.719548,...,-1.779400e-07,2.036389e-07,8.022884e-07,-2.538071e-07,1.863774e-06,4.875392e-07,3.212453e-07,6.347524e-07,6.815264e-07,7.782946e-07
1,10.305937,11.510157,-5.758058,-1.508590,-8.974911,-2.647319,-4.362800,4.050844,-0.830384,3.812462,...,9.048996e-07,-5.735345e-07,-6.721415e-07,8.550529e-07,3.789117e-07,1.700441e-07,-3.308357e-07,3.262595e-07,8.682251e-07,1.890846e-06
2,-8.348694,-0.363074,0.076656,-1.005099,1.440403,-0.442519,-1.781688,-1.407007,-0.044553,1.786873,...,4.977300e-07,8.914771e-07,-3.189962e-07,5.905976e-07,-7.565585e-07,-4.783064e-07,-1.011099e-06,1.576675e-06,-3.102626e-07,1.431196e-06
3,-1.383220,2.399253,-1.932302,3.060848,1.171404,-2.749633,0.390031,-4.638575,-0.636720,-3.605303,...,-2.019109e-07,7.114815e-07,3.220782e-07,4.626671e-08,3.813217e-08,1.569164e-06,-7.562271e-07,4.933081e-07,-3.309136e-07,1.466063e-07
4,-7.381315,1.570987,0.302512,-1.793322,-1.895764,1.705287,-2.147691,-0.547170,-0.804376,-2.348460,...,-8.186127e-07,9.035237e-08,5.769418e-07,-3.454535e-07,4.934978e-07,1.786420e-06,-5.246066e-07,2.030548e-07,-1.557495e-06,1.249375e-06


In [15]:
y=df['Level '] 

**Train Test Split**

In [16]:
# Train-Test-Split is performed

from sklearn.model_selection import train_test_split

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

((1220, 100), (524, 100))

In [17]:
print(y_train.value_counts(normalize=True)*100)
print("\n")
print(y_test.value_counts(normalize=True)*100)

2    45.327869
1    29.590164
3    25.081967
Name: Level , dtype: float64


2    45.229008
1    29.961832
3    24.809160
Name: Level , dtype: float64


**Model Building**

In [ ]:
def perceptron_classifier (X_train, y_train, X_test, y_test):
  p = Perceptron(random_state=150)
  p.fit(X_train, y_train)
  predictions_train = p.predict(X_train)
  predictions_test = p.predict(X_test)
  train_score = accuracy_score(predictions_train, y_train)
  #print("score on train data: ", train_score)
  test_score = f1_score(predictions_test, y_test, average='weighted' )
  print("Perceptron F1 score:", test_score)

In [ ]:
from sklearn.linear_model import Perceptron
perceptron_classifier(X_train, y_train, X_test, y_test) 

Perceptron F1 score: 0.4157088255124254


**Decision Tree**

In [18]:
clf_dt = DecisionTreeClassifier(max_depth=5, class_weight = "balanced")

In [19]:
clf_dt.fit(X_train,y_train)

DecisionTreeClassifier(class_weight='balanced', max_depth=5)

In [20]:
y_pred_train = clf_dt.predict(X_train)
y_pred_test = clf_dt.predict(X_test)

In [21]:
def classifcation_report_train_test(y_train, y_train_pred, y_test, y_test_pred):

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TRAIN DATA
            =========================================
            ''')
    print(classification_report(y_train, y_train_pred, digits=4))

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TEST DATA
            =========================================
            ''')
    print(classification_report(y_test, y_test_pred, digits=4))

In [22]:
scores = pd.DataFrame(columns=['Model', 'Train_Accuracy', 'Train_Recall', 'Train_Precision', 'Train_F1_Score', 
                               'Test_Accuracy', 'Test_Recall', 'Test_Precision', 'Test_F1_Score'])

def get_metrics(train_actual, train_predicted, test_actual, test_predicted, model_description, dataframe):

    train_accuracy  = accuracy_score(train_actual, train_predicted)
    train_recall    = recall_score(train_actual, train_predicted, average="weighted")
    train_precision = precision_score(train_actual, train_predicted, average="weighted")
    train_f1score   = f1_score(train_actual, train_predicted, average="weighted")
    
    test_accuracy   = accuracy_score(test_actual, test_predicted)
    test_recall     = recall_score(test_actual, test_predicted, average="weighted")
    test_precision  = precision_score(test_actual, test_predicted, average="weighted")
    test_f1score    = f1_score(test_actual, test_predicted, average="weighted")

    dataframe       = dataframe.append(pd.Series([model_description, 
                                                  train_accuracy, train_recall, train_precision, train_f1score,
                                                  test_accuracy, test_recall, test_precision, test_f1score],
                                                 index=scores.columns ), 
                                       ignore_index=True)

    return(dataframe)

In [23]:
scores = get_metrics(y_train, y_pred_train, y_test, y_pred_test, "DecisionTrees", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.440164,0.440164,0.62797,0.409161,0.282443,0.282443,0.374263,0.255656


**Random Forest**

In [24]:
clf_rf = RandomForestClassifier(class_weight = "balanced")

In [25]:
clf_rf.fit(X=X_train, y=y_train)

RandomForestClassifier(class_weight='balanced')

In [26]:
y_pred_train1 = clf_rf.predict(X_train)
y_pred_test1 = clf_rf.predict(X_test)

In [27]:
classifcation_report_train_test(y_train, y_pred_train1, y_test, y_pred_test1)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     1.0000    0.9972    0.9986       361
           2     1.0000    1.0000    1.0000       553
           3     0.9967    1.0000    0.9984       306

    accuracy                         0.9992      1220
   macro avg     0.9989    0.9991    0.9990      1220
weighted avg     0.9992    0.9992    0.9992      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3158    0.0764    0.1231       157
           2     0.4553    0.9030    0.6054       237
           3     0.2500    0.0308    0.0548       130

    accuracy                         0.4389       524
   macro avg     0.3404    0.3367    0.2611       524
weighted avg     0.3626    0.4389    0.3243       524



In [28]:
scores = get_metrics(y_train, y_pred_train1, y_test, y_pred_test1, "RandomForest", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.440164,0.440164,0.627970,0.409161,0.282443,0.282443,0.374263,0.255656
1,RandomForest,0.999180,0.999180,0.999183,0.999180,0.438931,0.438931,0.362576,0.324275


**AdaBoost**

In [ ]:
clf_adaboost =  AdaBoostClassifier()

In [ ]:
clf_adaboost.fit(X_train, y_train)

AdaBoostClassifier()

In [ ]:
y_pred_train2 = clf_adaboost.predict(X_train)
y_pred_test2 = clf_adaboost.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train2, y_test, y_pred_test2)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5572    0.4183    0.4778       361
           2     0.5529    0.7939    0.6518       553
           3     0.5935    0.3007    0.3991       306

    accuracy                         0.5590      1220
   macro avg     0.5679    0.5043    0.5096      1220
weighted avg     0.5644    0.5590    0.5370      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2707    0.2293    0.2483       157
           2     0.4237    0.5738    0.4875       237
           3     0.1429    0.0769    0.1000       130

    accuracy                         0.3473       524
   macro avg     0.2791    0.2934    0.2786       524
weighted avg     0.3082    0.3473    0.3197       524



In [ ]:
scores = get_metrics(y_train, y_pred_train2, y_test, y_pred_test2, "Adaboost", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.563934,0.563934,0.683444,0.496366,0.416031,0.416031,0.322004,0.321875
1,RandomForest,0.999180,0.999180,0.999183,0.999180,0.400763,0.400763,0.305608,0.313191
2,Adaboost,0.559016,0.559016,0.564365,0.536962,0.347328,0.347328,0.308166,0.319668


**Gradient Boosting Classifier**

In [ ]:
clf_gbm = GradientBoostingClassifier()

In [ ]:
clf_gbm.fit(X=X_train, y=y_train)

GradientBoostingClassifier()

In [ ]:
y_pred_train3 = clf_gbm.predict(X_train)
y_pred_test3 = clf_gbm.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train3, y_test, y_pred_test3)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9551    0.8255    0.8856       361
           2     0.7997    0.9892    0.8844       553
           3     0.9821    0.7190    0.8302       306

    accuracy                         0.8730      1220
   macro avg     0.9123    0.8445    0.8667      1220
weighted avg     0.8915    0.8730    0.8712      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2524    0.1656    0.2000       157
           2     0.4503    0.7257    0.5557       237
           3     0.2564    0.0769    0.1183       130

    accuracy                         0.3969       524
   macro avg     0.3197    0.3228    0.2914       524
weighted avg     0.3429    0.3969    0.3406       524



In [ ]:
scores = get_metrics(y_train, y_pred_train3, y_test, y_pred_test3, "GBM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.563934,0.563934,0.683444,0.496366,0.416031,0.416031,0.322004,0.321875
1,RandomForest,0.999180,0.999180,0.999183,0.999180,0.400763,0.400763,0.305608,0.313191
2,Adaboost,0.559016,0.559016,0.564365,0.536962,0.347328,0.347328,0.308166,0.319668
3,GBM,0.872951,0.872951,0.891455,0.871153,0.396947,0.396947,0.342894,0.340637


**SVM Model**

In [29]:
linear_svm = SVC(kernel='linear', C=1, class_weight = "balanced")

In [30]:
linear_svm.fit(X=X_train, y= y_train)

SVC(C=1, class_weight='balanced', kernel='linear')

In [31]:
y_pred_train4 = linear_svm.predict(X_train)
y_pred_test4 = linear_svm.predict(X_test)

In [32]:
classifcation_report_train_test(y_train, y_pred_train4, y_test, y_pred_test4)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4006    0.3906    0.3955       361
           2     0.5252    0.4141    0.4631       553
           3     0.3287    0.4641    0.3848       306

    accuracy                         0.4197      1220
   macro avg     0.4182    0.4229    0.4145      1220
weighted avg     0.4390    0.4197    0.4235      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2867    0.2611    0.2733       157
           2     0.4611    0.3755    0.4140       237
           3     0.2340    0.3385    0.2767       130

    accuracy                         0.3321       524
   macro avg     0.3273    0.3250    0.3213       524
weighted avg     0.3525    0.3321    0.3378       524



In [33]:
scores = get_metrics(y_train, y_pred_train4, y_test, y_pred_test4, "SVM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.440164,0.440164,0.627970,0.409161,0.282443,0.282443,0.374263,0.255656
1,RandomForest,0.999180,0.999180,0.999183,0.999180,0.438931,0.438931,0.362576,0.324275
2,SVM,0.419672,0.419672,0.439049,0.423465,0.332061,0.332061,0.352538,0.337777


**KNN**

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7)

In [ ]:
knn.fit(X=X_train, y=y_train)

KNeighborsClassifier(n_neighbors=7)

In [ ]:
y_pred_train5 = knn.predict(X_train)
y_pred_test5 = knn.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train5, y_test, y_pred_test5)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4688    0.4792    0.4740       361
           2     0.5898    0.7306    0.6527       553
           3     0.4578    0.2484    0.3220       306

    accuracy                         0.5352      1220
   macro avg     0.5055    0.4861    0.4829      1220
weighted avg     0.5209    0.5352    0.5169      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2606    0.2739    0.2671       157
           2     0.4357    0.5148    0.4720       237
           3     0.2405    0.1462    0.1818       130

    accuracy                         0.3511       524
   macro avg     0.3123    0.3116    0.3070       524
weighted avg     0.3348    0.3511    0.3386       524



In [ ]:
scores = get_metrics(y_train, y_pred_train5, y_test, y_pred_test5, "KNN", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.563934,0.563934,0.683444,0.496366,0.416031,0.416031,0.322004,0.321875
1,RandomForest,0.999180,0.999180,0.999183,0.999180,0.400763,0.400763,0.305608,0.313191
2,Adaboost,0.559016,0.559016,0.564365,0.536962,0.347328,0.347328,0.308166,0.319668
3,GBM,0.872951,0.872951,0.891455,0.871153,0.396947,0.396947,0.342894,0.340637
4,SVM,0.454098,0.454098,0.501532,0.284550,0.452290,0.452290,0.204566,0.281716
5,KNN,0.535246,0.535246,0.520897,0.516861,0.351145,0.351145,0.334819,0.338590


**GNB**

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

In [ ]:
nb.fit(X=X_train, y=y_train)

GaussianNB()

In [ ]:
y_pred_train6 = nb.predict(X_train)
y_pred_test6 = nb.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train6, y_test, y_pred_test6)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.3964    0.1856    0.2528       361
           2     0.4942    0.3834    0.4318       553
           3     0.2797    0.5686    0.3750       306

    accuracy                         0.3713      1220
   macro avg     0.3901    0.3792    0.3532      1220
weighted avg     0.4115    0.3713    0.3646      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2250    0.1146    0.1519       157
           2     0.4140    0.3249    0.3641       237
           3     0.2287    0.4538    0.3041       130

    accuracy                         0.2939       524
   macro avg     0.2892    0.2978    0.2734       524
weighted avg     0.3114    0.2939    0.2856       524



In [ ]:
scores = get_metrics(y_train, y_pred_train6, y_test, y_pred_test6, "GNB", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.563934,0.563934,0.683444,0.496366,0.416031,0.416031,0.322004,0.321875
1,RandomForest,0.999180,0.999180,0.999183,0.999180,0.400763,0.400763,0.305608,0.313191
2,Adaboost,0.559016,0.559016,0.564365,0.536962,0.347328,0.347328,0.308166,0.319668
3,GBM,0.872951,0.872951,0.891455,0.871153,0.396947,0.396947,0.342894,0.340637
4,SVM,0.454098,0.454098,0.501532,0.284550,0.452290,0.452290,0.204566,0.281716
5,KNN,0.535246,0.535246,0.520897,0.516861,0.351145,0.351145,0.334819,0.338590
6,GNB,0.371311,0.371311,0.411473,0.364583,0.293893,0.293893,0.311387,0.285626
